In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import random 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/Machine Hack - PlayStoreApp/Train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Machine Hack - PlayStoreApp/Test.csv')
submission = pd.read_csv('/content/gdrive/My Drive/Machine Hack - PlayStoreApp/Sample_Submission.csv')

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.nunique()

In [ ]:
train.Downloads.value_counts().plot.bar()

In [ ]:
test.head()

In [ ]:
objList_train = train.select_dtypes(include = "object").columns
print (objList)

In [ ]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList_train:
    train[feat] = le.fit_transform(train[feat].astype(str))

print (train.info())

In [ ]:
objList_test = test.select_dtypes(include = "object").columns
print (objList)

In [ ]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList_test:
    test[feat] = le.fit_transform(test[feat].astype(str))

print (test.info())

In [ ]:
X = train.drop('Downloads',1)
y = train.Downloads
test_final = test

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=7)

In [ ]:
from xgboost import XGBClassifier
model_2 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=1,
 colsample_bytree=0.8,
 seed=100,
 eval_metric='mlogloss'
 )
#model.fit(X_train, y_train)
model_2.fit(X_train, y_train, eval_metric='mlogloss', 
          eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)

In [ ]:
model_xgb=XGBClassifier(
 learning_rate =0.01,
 n_estimators=1100,
 max_depth=1,
 colsample_bytree=0.8,
 seed=100
 )
model_xgb.fit(X,y)

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(X=X,y=y,estimator=model_xgb,scoring='neg_log_loss',cv=10)

In [ ]:
score

In [ ]:
np.mean(score)

In [ ]:
y_pred = model_xgb.predict_proba(test_final)

In [ ]:
submission = pd.DataFrame(y_pred)

In [ ]:
submission.head()

In [ ]:
test=pd.read_csv('Test.csv')

In [ ]:
for i in range(submission.shape[0]):
    team1=test.iloc[i]['Team1']
    team2=test.iloc[i]['Team2']
    if submission.iloc[i][team1] > submission.iloc[i][team2]:
        submission.iloc[i][team1]=1-submission.iloc[i][team2]
    else:
        submission.iloc[i][team2]=1-submission.iloc[i][team1]

In [ ]:
submission.head()

In [ ]:
submission[submission<0.05]=0
submission[submission>0.95]=1

In [ ]:
submission.head()

In [ ]:
submission.to_excel('submission_final.xlsx',index=False)